In [25]:
import numpy as np
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [26]:
X, y = make_classification(n_samples=10000, n_features=64, n_informative=5, n_redundant=2, n_classes=10, random_state=42)

# Chuyển đổi sang DataFrame để dễ nhìn
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y


x_train, x_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

In [27]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
train_loader = DataLoader(TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train)), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test)), batch_size=32, shuffle=False)

(8000, 64) (2000, 64) (8000,) (2000,)


In [28]:
class NeuralNetwork(nn.Module):
  def __init__(self, input_size = 64, hidden_size1 = 128, hidden_size2 = 64, output_size = 10):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size1)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size1, hidden_size2)
    self.fc3 = nn.Linear(hidden_size2, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    out = self.sigmoid(out)
    return out

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=64, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)


In [29]:
model = NeuralNetwork()
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.float())
        loss = criterion(output, target.long())  # Use target as long for CrossEntropyLoss
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in test_loader:
            output = model(data.float())
            _, predicted = torch.max(output.data, 1)  # Use torch.max to get predicted class
            total += target.size(0)
            correct += (predicted == target).sum().item()
        if (epoch+1) % 10 == 0:
          print(f"Epoch {epoch+1} Accuracy: {correct/total}")

Epoch 10 Accuracy: 0.5035
Epoch 20 Accuracy: 0.539
Epoch 30 Accuracy: 0.5485
Epoch 40 Accuracy: 0.5475
Epoch 50 Accuracy: 0.5455
Epoch 60 Accuracy: 0.544
Epoch 70 Accuracy: 0.5425
Epoch 80 Accuracy: 0.538
Epoch 90 Accuracy: 0.548
Epoch 100 Accuracy: 0.5435
